In [1]:
# import torch

# from learning.network.sac import *

# policy = Policy()
# qf = QFunction()

# image_feature = torch.randn(5, 512)
# object_feature = torch.randn(5, 512)
# action = torch.randn(5, 2)

# policy(image_feature, object_feature)

# qf(image_feature, object_feature, action)

In [2]:
import random
import numpy as np
import os
from PIL import Image

import torch
torch.__version__

'1.11.0+cu113'

In [3]:
import getpass
user = getpass.getuser()
print(user)

yizhou


In [4]:
usd_path = f"omniverse://localhost/Users/{user}/uva_sac.usd"

In [5]:
from omni.isaac.kit import SimulationApp    
simulation_app = SimulationApp({"headless": True, "open_usd": usd_path,  "livesync_usd": usd_path}) 

Starting kit application with the fillowing args:  ['/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/exts/omni.isaac.kit/omni/isaac/kit/simulation_app.py', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/apps/omni.isaac.sim.python.kit', '--/app/tokens/exe-path=/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/kit', '--/persistent/app/viewport/displayOptions=3094', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=True', '--ext-folder', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/exts', '--ext-folder', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/apps', '--/physics/cudaDevice=0', '--portable', '--no-window']
Passing the following args to the base kit application:  ['-f', '/home/yizhou/.local/share/jupyter/runtime/kernel-b02b60ab-c877-

OmniAssetFileFormat


Warp initialized:
   Version: 0.2.2
   CUDA device: NVIDIA GeForce RTX 3090
   Kernel cache: /home/yizhou/.cache/warp/0.2.2
[2.763s] [ext: omni.kit.menu.create-1.0.2] startup
[2.765s] [ext: omni.mdl-0.1.0] startup
[2.790s] [ext: omni.kit.window.file_importer-1.0.4] startup
[2.792s] [ext: omni.kit.window.file_exporter-1.0.4] startup
[2.794s] [ext: omni.kit.material.library-1.3.10] startup
[2.800s] [ext: omni.kit.window.drop_support-1.0.0] startup
[2.802s] [ext: omni.kit.window.file-1.3.16] startup
[2.807s] [ext: omni.kit.context_menu-1.3.9] startup
[2.814s] [ext: omni.kit.window.property-1.6.3] startup
[2.816s] [ext: omni.kit.window.content_browser-2.4.28] startup
[2.828s] [ext: omni.kit.widget.stage-2.6.15] startup
[2.834s] [ext: omni.isaac.version-1.0.0] startup
[2.835s] [ext: omni.kit.property.usd-3.14.9] startup
[2.873s] [ext: omni.kit.viewport.legacy_gizmos-1.0.0] startup
[2.879s] [ext: omni.hydra.rtx-0.1.0] startup
[2.896s] [ext: omni.renderer-rtx-0.0.0] startup
[2.898s] [ext: omn

In [6]:
from uva_env import UvaEnv
env = UvaEnv()

In [7]:
from task.utils import add_scene_default
add_scene_default()

In [8]:
print(list(env.stage.TraverseAll()))

[Usd.Prim(</physicsScene>), Usd.Prim(</World>), Usd.Prim(</World/defaultLight>), Usd.Prim(</World/groundPlane>), Usd.Prim(</World/groundPlane/CollisionMesh>), Usd.Prim(</World/groundPlane/CollisionPlane>), Usd.Prim(</World/layout>), Usd.Prim(</World/layout/structure>), Usd.Prim(</World/layout/structure/WallTop_2>), Usd.Prim(</World/layout/structure/WallBottom_3>), Usd.Prim(</World/layout/structure/WallInner_4>), Usd.Prim(</World/layout/structure/WallOuter_5>), Usd.Prim(</World/layout/structure/Baseboard_6>), Usd.Prim(</World/layout/structure/Baseboard_7>), Usd.Prim(</World/layout/structure/WallTop_8>), Usd.Prim(</World/layout/structure/WallBottom_9>), Usd.Prim(</World/layout/structure/WallInner_10>), Usd.Prim(</World/layout/structure/WallOuter_11>), Usd.Prim(</World/layout/structure/Baseboard_12>), Usd.Prim(</World/layout/structure/Baseboard_13>), Usd.Prim(</World/layout/structure/WallInner_14>), Usd.Prim(</World/layout/structure/Baseboard_15>), Usd.Prim(</World/layout/structure/Basebo

In [9]:
env.clean()
env.world.step(render=True)

# Config

In [10]:
from task.config import DATA_PATH
task_type = "Table"
side_choice = "Border"
asset_id = 0
load_nucleus = True

traj_id = 0
image_folder = os.path.join(DATA_PATH, task_type, side_choice, str(traj_id))

# Scene

In [11]:
from task.scene import ArrangeScene
scene = ArrangeScene(task_type, side_choice, asset_id, "/World/base", traj_id, load_nucleus)
env.scene = scene

In [12]:
# base
scene.add_base_asset()

asset_prim: omniverse://localhost/Users/yizhou/Asset/T/Appleseed/Appleseed_CoffeeTable.usd


In [13]:
# add room
# scene.add_room()

In [14]:
env.world.step(render=True)

# Reward

In [15]:
from uv.reward import Rewarder

In [16]:
rewarder = Rewarder(env.world)
env.rewarder = rewarder

# Render

In [17]:
from render.helper import RenderHelper
render = RenderHelper(task_type, side_choice)

render.add_task_cameras()
render.set_cameras()

xform_mat:  ( (-1, 0, 0, 0), (0, -1, 0, 0), (0, 0, 1, 0), (0, 56, 200, 1) )


In [18]:
# get images
env.world.render()
env.world.render()
images = render.get_images()
render.save_rgb(images[0]['rgb'], image_folder, "0")

In [19]:
pause

NameError: name 'pause' is not defined

# Trajectory

In [20]:
step = 1

for step in range(5):

    env.add_scene_obj(mode = "random")

    x, y = np.tanh(np.random.randn()), np.tanh(np.random.randn())

    env.put_last_object((x, y)) 
    env.world.step(render=True)

    env.register_last_object()
    env.world.step(render=True)

    # env.scene.objects

    env.calculate_last_reward()

    env.world.reset()
    env.world.render()

    # get images
    env.world.render()
    env.world.render()
    images = render.get_images()
    render.save_rgb(images[0]['rgb'], image_folder, str(step))

loading asset from omni nucleus
object_folder ['AbstractFace_Small.usd', 'Abstract_Face.usd', 'AntiquelVaseSmall.usd', 'Bowl_Footed.usd', 'BrassVase.usd', 'CellVase01.usd', 'CellVase02.usd', 'CellVase03.usd', 'CellVase04.usd', 'CellVase05.usd', 'Earthenware01.usd', 'Earthenware02.usd', 'Earthenware03.usd', 'Knit.usd', 'Prime2Layer.usd', 'Prime3Layer.usd', 'Prime_DoubleGourd.usd', 'Prime_Large.usd', 'RecycledGlassVase_Medium.usd', 'RecycledGlassVase_Small.usd', 'Seahorn.usd', 'Vase_Cylinder.usd', 'Vase_D.usd', 'japan_vase.usd']
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Vase/RecycledGlassVase_Medium.usd
/World/objects/Vase begin_translation (64.10908508300781, 80.4120864868164, 44.997249603271484) timecode 0.0 end_translation (64.11054229736328, 80.34156799316406, 43.97065734863281)
/World/objects/Vase begin_translation (64.10908508300781, 80.4120864868164, 45) timecode 0.0 end_translation (64.11114501953125, 80.34589385986328, 44.00154113769531)
loading asset from omni nucl

In [21]:
# Record

record = env.scene.get_scene_data()
env.scene.save_scene_data()
print("record: ", record)


record:  {'base': {'task_choice': 'Table', 'side_choice': 'Border', 'base_asset_id': 0, 'base_file_path': 'T/Appleseed/Appleseed_CoffeeTable.usd', 'transform': {'translation': [0.0, 37.46500015258789, -9.5367431640625e-06], 'rotation': [1, 0, 0, 0], 'scale': 1}}, 'objects': [{'type': 'Vase', 'name': 'RecycledGlassVase_Medium.usd', 'file_path': 'I/Vase/RecycledGlassVase_Medium.usd', 'image_path': 'I/Vase/.thumbs/256x256/RecycledGlassVase_Medium.usd.png', 'prim_path': '/World/objects/Vase', 'xform_name': 'Vase', 'transform': {'translation': [64.10908508300781, 80.4120864868164, 45.0], 'rotation': [1, 0, 0, 0], 'scale': 1.0}, 'action': (0.6410908186214138, 0.8041208608732835), 'reward': {'affordance': 1.029012457992995, 'perturbation': 1.000652690455688}}, {'type': 'Flatware', 'name': 'Spoon_Big.usd', 'file_path': 'I/Flatware/Spoon_Big.usd', 'image_path': 'I/Flatware/.thumbs/256x256/Spoon_Big.usd.png', 'prim_path': '/World/objects/Flatware', 'xform_name': 'Flatware', 'transform': {'transl

# Reset (env clean)

In [ ]:
env.scene.objects

In [ ]:
env.clean()

In [ ]:
env.step(render = True)

In [ ]:
image_folder